Importing modules

In [60]:
import numpy as np
import cv2
import glob


# This is a value for the termination criterion of the subpixel corner localizer
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

Creation of the 3D point coordinates and loading images

In [61]:
# Here you have 3D points for the checkerboard (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
print(objp)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# Here you load the image files
images = glob.glob('left/left*.jpg')

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


Cell 1

In [62]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
print(len(images))
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)
        img = cv2.drawChessboardCorners(img, (7,6), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)



cv2.destroyAllWindows()




13


Cell 2

In [71]:
def scaling_matrix(s):
    # Scales are given in numpy array s
    # s=np.array([s_x,s_y,s_z])
    
    # Write the scaling matrix here
    #******************************
    S = np.identity(4)
    for i in range(3):
        S[i][i] = s[i]
    return S

# Translation matrix
def translation_matrix(t):
    # Translation vector is given in numpy array t
    # t=np.array([t_x,t_y,t_z])
    
    # Write the translation matrix here
    #******************************
    T = np.identity(4)
    T[:3,3] = t
    return T

# Rotation matrix
def rotation_matrix(theta, v):
    # Axis for rotation (unit norm vector) is given 
    # is given in numpy array v
    # v=np.array([v_x,v_y,v_z])
    # Rotation angle is given in theta
    
    # Write the rotation matrix here
    st = np.sin(theta)
    ct = np.cos(theta)
    o_ct = 1-ct
    vx2= v[0]**2
    vy2= v[1]**2
    vz2= v[2]**2
    vxy = v[0]*v[1]
    vxz = v[0]*v[2]
    vyz= v[1]*v[2]
    # Rotation, Rodriguez formula :
    R = [[ct+vx2*o_ct, vxy*o_ct-v[2]*st, vxz*o_ct+v[1]*st],
         [vxy*o_ct+v[2]*st, ct+vy2*o_ct, vyz*o_ct-v[0]*st],
         [vxz*o_ct-v[1]*st, vyz*o_ct+v[0]*st, ct+vz2*o_ct]]
    R_v = np.identity(3)
    R_v[:3, :3] = R
    
    return R_v

In [87]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

#the rvec is a vector representing the rotation axis, and its length encodes the angle in radians to rotate. 
#the Rodrigues method turns such a rotation vector into a 3x3 matrix.

# in total we have 11 pictures the value of ret ~0.1 so we have a good calibration

# To find the rotation matrix for each picture we simply applies the rodriguez formula seen in TD2.

# once we have the rotation matrix of ONE rvecs vector(because rvecs is a vector of rotations vectors, for each of our
# 11 patenrs). We use the rodriguez formula to find the rotation matrix, and once we have it we add the translation
# vector (in tvecs which represtents a vector of translation vectors for each pattern/picture), to create our 
# extrinsic matrix for one pattern. And we rpeate this procedure for each pattern



# test

angle = len(rvecs[0])*0.01 # pourquoi *0.01 ? parce que c'étais marque dans le TD2 et j'ai copié/collé

rotation_0,_ = cv2.Rodrigues(rvecs[0])
print("det rotation_0 =",np.linalg.det(rotation_0))
M_ext_0 = np.array(np.hstack((rotation_0,tvecs[0])),dtype=float)
print(M_ext_0)

angle = len(rvecs[1])*0.01

rotation_1,_= cv2.Rodrigues(rvecs[1])
print("det rotation_1 =",np.linalg.det(rotation_1))
M_ext_1 = np.array(np.hstack((rotation_1,tvecs[1])),dtype=float)
print(M_ext_1)

angle = len(rvecs[2])*0.01

rotation_2,_ = cv2.Rodrigues( rvecs[2])
print("det rotation_2 =",np.linalg.det(rotation_2))
M_ext_2 = np.array(np.hstack((rotation_2,tvecs[2])),dtype=float)
print(M_ext_2)


angle = len(rvecs[3])*0.01

rotation_3,_= cv2.Rodrigues(rvecs[3])
print("det rotation_3 =",np.linalg.det(rotation_3))
M_ext_3 = np.array(np.hstack((rotation_3,tvecs[3])),dtype=float)
print(M_ext_3)

angle = len(rvecs[4])*0.01

rotation_4,_ = cv2.Rodrigues(rvecs[4])
print("det rotation_4 =",np.linalg.det(rotation_4),"\n")
M_ext_4 = np.array(np.hstack((rotation_4,tvecs[4])),dtype=float)
print(M_ext_4)

# ...



det rotation_0 = 1.0000000000000004
[[-9.61727018e-01 -1.01232220e-02  2.73822321e-01  3.79739146e+00]
 [-3.51069793e-02 -9.86528861e-01 -1.59775803e-01  8.98950175e-01]
 [ 2.71751068e-01 -1.63273781e-01  9.48416063e-01  1.48593055e+01]]
det rotation_1 = 0.9999999999999999
[[-5.72680869e-03  9.97399430e-01  7.18441364e-02 -2.89985494e+00]
 [-9.28630323e-01 -3.19567624e-02  3.69627499e-01  2.62066993e+00]
 [ 3.70962163e-01 -6.45998576e-02  9.26398366e-01  1.05769926e+01]]
det rotation_2 = 0.9999999999999999
[[-0.30887909  0.9501694   0.04209285 -1.22339632]
 [-0.83758618 -0.25077985 -0.48534406  3.52139948]
 [-0.45060304 -0.18516902  0.87330941 15.67311789]]
det rotation_3 = 0.9999999999999999
[[ 0.14620389 -0.89550824 -0.4203444   1.81888151]
 [ 0.96337408  0.22544365 -0.14520858 -4.2642919 ]
 [ 0.22479946 -0.38371885  0.89567017 12.45728517]]
det rotation_4 = 0.9999999999999998 

[[ 0.09449387  0.97566706  0.19785017 -2.12419184]
 [-0.75316188  0.20002654 -0.62668698  1.84550818]
 [-0

Camera matrix refinement

In [79]:
img = cv2.imread('left/left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

Cell 3

In [81]:
# 1
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult1.png',dst)

# 2
mapx,mapy = cv2.initUndistortRectifyMap(mtx,dist,None,newcameramtx,(w,h),5)
dst = cv2.remap(img,mapx,mapy,cv2.INTER_LINEAR)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult2.png',dst)

True

Cell 4

this cell computes the rate of error of finding corners and calibrating pattern. When this value approaches 0
this will mean that there are no errors and the calibration was perfect. This implies that cv.findChessboardCorners(...) shall alwayes return true.

In [88]:
tot_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    tot_error += error

print("total error: ", tot_error/len(objpoints))

total error:  0.023686000375385673
